In [1]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, max, min, count, broadcast, desc, asc, when, lit, round
from pyspark.sql.types import StringType, IntegerType, FloatType

In [2]:
def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


14:27:59.884 [Thread-3] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))


In [5]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))




flight_row_count = df_flights.count()

airline_row_count = df_airline.count()

airports_row_count = df_airports.count()

14:28:00.097 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
14:28:00.108 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
14:28:00.124 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@67eb9272{/SQL,null,AVAILABLE,@Spark}
14:28:00.125 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@d7eadb7{/SQL/json,null,AVAILABLE,@Spark}
14:28:00.126 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@2eef4296{/SQL/execution,null,AVAILABLE,@Spark}
14:28:00.127 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@12422f79{/SQL/execution/json,null,AVAILABLE,@Spark}
14:28:00.141 [Thread

14:28:02.709 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3010 bytes result sent to driver
14:28:02.723 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 558 ms on 172.23.57.81 (executor driver) (1/1)
14:28:02.726 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
14:28:02.732 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.772 s
14:28:02.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
14:28:02.736 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

14:28:02.923 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_0_piece0 on 172.23.57.81:42041 in memory (size: 36.9 KiB, free: 434.4 MiB)
14:28:03.627 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.InMemoryFileIndex - It took 4 ms to list leaf files for 1 paths.
14:28:03.691 [Thread-3] INFO  org.apache.spark.SparkContext - Starting job: parquet at NativeMethodAccessorImpl.java:0
14:28:03.693 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got job 1 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
14:28:03.694 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ResultStage 1 (parquet at NativeMethodAccessorImpl.java:0)
14:28:03.694 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
14:28:03.694 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: Lis

14:28:05.882 [Executor task launch worker for task 4.0 in stage 3.0 (TID 7)] INFO  org.apache.spark.executor.Executor - Finished task 4.0 in stage 3.0 (TID 7). 2222 bytes result sent to driver
14:28:05.884 [Executor task launch worker for task 0.0 in stage 3.0 (TID 3)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 3.0 (TID 3). 2222 bytes result sent to driver
14:28:05.884 [Executor task launch worker for task 3.0 in stage 3.0 (TID 6)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 3.0 (TID 6). 2222 bytes result sent to driver
14:28:05.885 [Executor task launch worker for task 6.0 in stage 3.0 (TID 9)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 3.0 (TID 9). 2222 bytes result sent to driver
14:28:05.887 [Executor task launch worker for task 1.0 in stage 3.0 (TID 4)] INFO  org.apache.spark.executor.Executor - Finished task 1.0 in stage 3.0 (TID 4). 2179 bytes result sent to driver
14:28:05.891 [Executor task launch 

14:28:06.335 [dag-scheduler-event-loop] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_7 stored as values in memory (estimated size 17.0 KiB, free 434.1 MiB)
14:28:06.338 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_3_piece0 on 172.23.57.81:42041 in memory (size: 34.5 KiB, free: 434.4 MiB)
14:28:06.338 [dag-scheduler-event-loop] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_7_piece0 stored as bytes in memory (estimated size 7.8 KiB, free 434.1 MiB)
14:28:06.339 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Added broadcast_7_piece0 in memory on 172.23.57.81:42041 (size: 7.8 KiB, free: 434.4 MiB)
14:28:06.342 [dag-scheduler-event-loop] INFO  org.apache.spark.SparkContext - Created broadcast 7 from broadcast at DAGScheduler.scala:1585
14:28:06.343 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting 1 missing tasks from Shuf

In [6]:
try:
    #avg per fly
    avg_flight = df_flights.join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                        .groupBy(df_flights.AIRLINE, df_airline.AIRLINE) \
                        .agg(round(avg(df_flights.DISTANCE), 2).alias("avg_DISTANCE")) \
                        .orderBy(desc("avg_DISTANCE")) \
                        .select(df_airline.AIRLINE.alias("AIRLINE_Name"), "avg_DISTANCE")
except Exception as error:
    logger.error("Error variable avg_flight:" + str(error))


In [7]:
try:
    #how many times a flight visit an airport
    airline_in_airport = df_flights.join(broadcast(df_airports), df_flights.ORIGIN_AIRPORT == df_airports.IATA_CODE) \
                                .join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                                .repartition(df_flights.DESTINATION_AIRPORT) \
                                .groupBy(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE) \
                                    .agg(count(df_flights.AIRLINE).alias("Count_visit_per_airline")) \
                                .orderBy(asc(df_flights.DESTINATION_AIRPORT)) \
                                .select(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE,"Count_visit_per_airline") \
                                .limit(100)
except Exception as error:
    logger.error("Error variable airline_in_airport:" + str(error))


In [8]:
try:
    #mention how many times an airline visit a destination and canceled this arrival
    flights_per_cancell = df_flights.select(col("AIRLINE"), col("CANCELLED"), col("DESTINATION_AIRPORT")) \
                                .filter(col("CANCELLED") == "1") \
                                .repartition(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                .groupBy(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                    .agg(count(col("AIRLINE")).alias("count_airlines_cancel")) \
                                .orderBy(desc("DESTINATION_AIRPORT")) \
                                .limit(100)
except Exception as error:
    logger.error("Error variable flights_per_cancell:" + str(error))


In [9]:
try:
    #i create this variable because i need to know how many airports have flights but are not in the list of airports
    airport_notin_thelist = df_flights.join(broadcast(df_airports), df_flights.DESTINATION_AIRPORT == df_airports.IATA_CODE, "left_anti") \
                                        .select("DESTINATION_AIRPORT") \
                                        .repartition("DESTINATION_AIRPORT") \
                                        .groupBy("DESTINATION_AIRPORT").agg(count("*").alias("Count_airports"))
except Exception as error:
    logger.error("Error variable airport_notin_thelist:" + str(error))


In [10]:
#Create parquet files

try:
    avg_flight.write.parquet(config["Parquet_file"]["avg_flight"], mode="overwrite")

    airline_in_airport.write.parquet(config["Parquet_file"]["airline_in_airport"], mode="overwrite")

    flights_per_cancell.write.parquet(config["Parquet_file"]["flights_per_cancell"], mode="overwrite")

    airport_notin_thelist.write.parquet(config["Parquet_file"]["airport_notin_thelist"], mode="overwrite")
except Exception as error:
    logger.error("Error write parquet files:" + str(error))


14:28:07.744 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(AIRLINE)
14:28:07.745 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(AIRLINE#4)
14:28:07.751 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(IATA_CODE)
14:28:07.752 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(IATA_CODE#62)
14:28:07.816 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_10_piece0 on 172.23.57.81:42041 in memory (size: 7.8 KiB, free: 434.4 MiB)
14:28:07.822 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_11_piece0 on 172.23.57.81:42041 in memory (size: 5.9 KiB, free: 434.4 MiB)
14:28:07.903 [broadcast-exchange-0] INFO  org.apache.spark.sql.catalyst.expressions.codegen.C

14:28:10.138 [Executor task launch worker for task 7.0 in stage 13.0 (TID 24)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 13.0 (TID 24). 3926 bytes result sent to driver
14:28:10.140 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 7.0 in stage 13.0 (TID 24) in 1451 ms on 172.23.57.81 (executor driver) (1/8)
14:28:10.157 [Executor task launch worker for task 0.0 in stage 13.0 (TID 17)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 13.0 (TID 17). 3926 bytes result sent to driver
14:28:10.158 [Executor task launch worker for task 3.0 in stage 13.0 (TID 20)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 13.0 (TID 20). 3926 bytes result sent to driver
14:28:10.160 [Executor task launch worker for task 6.0 in stage 13.0 (TID 23)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 13.0 (TID 23). 3926 bytes result sent to driver
14:28:10.162 [task-result-getter-3

14:28:10.602 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 12.43469 ms
14:28:10.617 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Getting 8 (11.2 KiB) non-empty blocks including 8 (11.2 KiB) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
14:28:10.617 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Started 0 remote fetches in 1 ms
14:28:10.639 [Executor task launch worker for task 0.0 in stage 17.0 (TID 26)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 17.0 (TID 26). 6433 bytes result sent to driver
14:28:10.641 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 17.0 (TID 26) in 64 ms on 172.23.57.81 (exec

14:28:12.698 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_12_piece0 on 172.23.57.81:42041 in memory (size: 34.6 KiB, free: 434.2 MiB)
14:28:12.741 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_23_piece0 on 172.23.57.81:42041 in memory (size: 6.3 KiB, free: 434.2 MiB)
14:28:12.762 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_15_piece0 on 172.23.57.81:42041 in memory (size: 34.7 KiB, free: 434.3 MiB)
14:28:12.836 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_14_piece0 on 172.23.57.81:42041 in memory (size: 686.0 B, free: 434.3 MiB)
14:28:13.580 [Executor task launch worker for task 6.0 in stage 23.0 (TID 36)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 23.0 (TID 36). 2451 bytes result sent to driver
14:28:13.582 [task-result-getter-3] INFO  org.apa

14:28:14.332 [Executor task launch worker for task 7.0 in stage 23.0 (TID 37)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 23.0 (TID 37). 2494 bytes result sent to driver
14:28:14.333 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 7.0 in stage 23.0 (TID 37) in 2379 ms on 172.23.57.81 (executor driver) (2/8)
14:28:14.344 [Executor task launch worker for task 5.0 in stage 23.0 (TID 35)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 23.0 (TID 35). 2451 bytes result sent to driver
14:28:14.345 [Executor task launch worker for task 2.0 in stage 23.0 (TID 32)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 23.0 (TID 32). 2451 bytes result sent to driver
14:28:14.345 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 23.0 (TID 35) in 2392 ms on 172.23.57.81 (executor driver) (3/8)
14:28:14.347 [Executor task launch worker for task 3.

...

14:28:15.865 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ShuffleMapStage 25 (parquet at NativeMethodAccessorImpl.java:0) finished in 1.145 s
14:28:15.865 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - looking for newly runnable stages
14:28:15.866 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - running: Set()
14:28:15.866 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - waiting: Set(ResultStage 26)
14:28:15.866 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - failed: Set()
14:28:15.866 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ResultStage 26 (MapPartitionsRDD[62] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
14:28:15.892 [dag-scheduler-event-loop] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_29 stored as values in memory (estimated size 244.8 KiB, free 4

14:28:16.318 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_30 stored as values in memory (estimated size 200.5 KiB, free 429.0 MiB)
14:28:16.336 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_30_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 429.0 MiB)
14:28:16.337 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Added broadcast_30_piece0 in memory on 172.23.57.81:42041 (size: 34.8 KiB, free: 434.1 MiB)
14:28:16.338 [Thread-3] INFO  org.apache.spark.SparkContext - Created broadcast 30 from parquet at NativeMethodAccessorImpl.java:0
14:28:16.339 [Thread-3] INFO  org.apache.spark.sql.execution.FileSourceScanExec - Planning scan with bin packing, max size: 21880221 bytes, open cost is considered as scanning 4194304 bytes.
14:28:16.356 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Registering RDD 66 (parquet at NativeMethodAccessorImpl.java:0) as i

14:28:17.155 [Executor task launch worker for task 3.0 in stage 27.0 (TID 52)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 27.0 (TID 52). 2296 bytes result sent to driver
14:28:17.157 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 3.0 in stage 27.0 (TID 52) in 783 ms on 172.23.57.81 (executor driver) (3/8)
14:28:17.160 [Executor task launch worker for task 2.0 in stage 27.0 (TID 51)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 27.0 (TID 51). 2296 bytes result sent to driver
14:28:17.161 [Executor task launch worker for task 5.0 in stage 27.0 (TID 54)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 27.0 (TID 54). 2296 bytes result sent to driver
14:28:17.162 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 2.0 in stage 27.0 (TID 51) in 789 ms on 172.23.57.81 (executor driver) (4/8)
14:28:17.162 [task-result-getter-3] INFO  org.apache.sp

14:28:17.888 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileFormatWriter - Write Job 7bf799cf-19db-4410-8e72-385abca53345 committed. Elapsed time: 175 ms.
14:28:17.888 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileFormatWriter - Finished processing stats for write job 7bf799cf-19db-4410-8e72-385abca53345.
14:28:17.934 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: 
14:28:17.934 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: 
14:28:17.936 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(IATA_CODE)
14:28:17.936 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(IATA_CODE#66)
14:28:17.971 [broadcast-exchange-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_33 stored as values in memory (estimated size 200.2 KiB, free

14:28:18.943 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.parquet.ParquetUtils - Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
14:28:18.944 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
14:28:18.945 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
14:28:18.945 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol - Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
14:28:18.945 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
14:28:18.945 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary f